<a href="https://colab.research.google.com/github/dmd8a/Climate-Change-Natural-Disasters/blob/main/Homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
filename = 'netflix_titles.csv'
df = pd.read_csv('/content/drive/MyDrive/netflix_titles.csv')



Mounted at /content/drive


**Question 1**. First, try to find all duplicate entries.  I began by searching for duplicate titles, and found none.  There are multiple entries with duplicate directors, as expected, but none with the same title (hence of course none with the same full row).

In [6]:
print(df.shape) # 7787 rows and 12 columns

# how many entries have the same title?  apparently none

duplicateTitles = df[df.duplicated(['title'])]
print(duplicateTitles)

# proof of concept: how many entries have the same director?  many

duplicateDirectors = df[df.duplicated(['director'])]
print(duplicateDirectors)
print(duplicateDirectors.count(axis=0)) 

(7787, 12)
Empty DataFrame
Columns: [show_id, type, title, director, cast, country, date_added, release_year, rating, duration, listed_in, description]
Index: []
     show_id  ...                                        description
11       s12  ...  In this dark alt-history thriller, a naïve law...
16       s17  ...  As a psychology professor faces Alzheimer's, h...
19       s20  ...  Mixing old footage with interviews, this is th...
24       s25  ...  Seiya and the Knights of the Zodiac rise again...
26       s27  ...  This docuseries takes a deep dive into the luc...
...      ...  ...                                                ...
7778   s7779  ...  Looking to survive in a world taken over by zo...
7779   s7780  ...  An assortment of talent takes the stage for a ...
7780   s7781  ...  A drug dealer starts having doubts about his t...
7784   s7785  ...  In this documentary, South African rapper Nast...
7785   s7786  ...  Dessert wizard Adriano Zumbo looks for the nex...

[3737 row

Next, I made all titles lower-case to find any hidden duplicates.  Still none.

In [7]:
# make the title all lower-case to see if we're missing any duplicates

lowercaseTitle = df['title'].str.lower()
print(lowercaseTitle.tail()) # to be sure the code is what it should be
#duplicateTitleslc = lowercaseTitle[lowercaseTitle.duplicated()]
#print(duplicateTitleslc)

7782                                       zozo
7783                                     zubaan
7784                          zulu man in japan
7785                      zumbo's just desserts
7786    zz top: that little ol' band from texas
Name: title, dtype: object


**Question 2**.  Find the percentage of missing data in each column.

In [9]:
counts=df.count() # gives the number of non-missing entries in each column
proportion_missing = 1 - (counts / len(df))
percent_missing = proportion_missing*100
print(percent_missing)


show_id          0.000000
type             0.000000
title            0.000000
director        30.679337
cast             9.220496
country          6.510851
date_added       0.128419
release_year     0.000000
rating           0.089893
duration         0.000000
listed_in        0.000000
description      0.000000
dtype: float64


This is another way to compute the percentage of missing data.  I'm including it here for my own reference.

In [10]:
df_missing = df.isna()
df_prop_missing=df_missing.mean()
percent_missing=df_prop_missing*100
print(percent_missing)

show_id          0.000000
type             0.000000
title            0.000000
director        30.679337
cast             9.220496
country          6.510851
date_added       0.128419
release_year     0.000000
rating           0.089893
duration         0.000000
listed_in        0.000000
description      0.000000
dtype: float64


**Question 3**.  Extract date information, adding three columns to the data frame.

In [12]:
 #print(df['date_added']) #first, figure out how the dates are entered - looks 
 # like day space month comma space year

date_split1=df['date_added'].str.split(',', expand=True) #splits off the month / day
# from the year, where year is in column=1
#print(date_split1.head())

date_split2=date_split1[0].str.split(' ',expand=True) # splits the month from the day,
# where month is in column 0 and day is in column 1

month = date_split2[0] #creates a data frame just with the month
day = date_split2[1] # creates a data frame just with the day
year = date_split1[1] # creates a data frame just with the year
df2 = pd.concat([df,year,month], axis=1) # add the year and month to the data frame
df2.rename(columns = {0: 'date_added_month', 1: 'date_added_year'}, inplace = True) # renamed the year and month columns
#print(df2.head())

df3 = pd.concat([df2,day], axis=1) # add the day column
df3.rename(columns = {1: 'date_added_day'}, inplace = True) 
print(df3.head())



  show_id     type  title  ... date_added_year date_added_month date_added_day
0      s1  TV Show     3%  ...            2020           August             14
1      s2    Movie   7:19  ...            2016         December             23
2      s3    Movie  23:59  ...            2018         December             20
3      s4    Movie      9  ...            2017         November             16
4      s5    Movie     21  ...            2020          January              1

[5 rows x 15 columns]


**Question 4**. Separate the duration column into minutes or Seasons.

In [14]:
seasondf = df3['duration'].loc[df3['duration'].str.contains('Season', case=False)]
# creates a data frame of entries from the duration column that had seasons
#print(seasondf.head())

minutesdf = df3['duration'].loc[df3['duration'].str.contains('min', case=False)] 
# creates a data frame of entries from the duration column that had minutes
#print(minutesdf.head())

newdf3=df3.drop(['duration'], axis=1) # drops the original 'duration' column

df4 = pd.concat([newdf3, seasondf], axis=1) # tacks on a season column, if the duration
# column had seasons
df4.rename(columns = {'duration': 'seasons'}, inplace = True) # renames the added column
#print(df4.head())

df5 = pd.concat([df4, minutesdf], axis=1)
df5.rename(columns = {'duration': 'minutes'}, inplace = True) # renames the minutes column
print(df5.head()) # this is the final answer

print(df5.columns) # just checking that we have all the columns





  show_id     type  title  ... date_added_day    seasons  minutes
0      s1  TV Show     3%  ...             14  4 Seasons      NaN
1      s2    Movie   7:19  ...             23        NaN   93 min
2      s3    Movie  23:59  ...             20        NaN   78 min
3      s4    Movie      9  ...             16        NaN   80 min
4      s5    Movie     21  ...              1        NaN  123 min

[5 rows x 16 columns]
Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'listed_in', 'description', 'date_added_year',
       'date_added_month', 'date_added_day', 'seasons', 'minutes'],
      dtype='object')
